In [ ]:
from pathlib import Path
import pathlib
from google.colab import drive
import google
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import statistics as st
from importlib_metadata import version
import sklearn as sk

In [ ]:
print(sk.__version__)

1.6.1


In [ ]:
print(pd.__version__)
print(np.__version__)
print(version("matplotlib"))
print(version("scipy"))
print(version("google"))

In [ ]:
drive.mount("/content/drive", force_remount=True)

In [ ]:
sudeste_files = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/Estados_do_sudeste/").glob("*.csv"))
sul_files = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/Estados_do_sul/").glob("*.csv"))
centro_oeste_files = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/Estados_do_centro_oeste/").glob("*.csv"))
nordeste_files = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/Estados_do_nordeste/").glob("*.csv"))
produtividade_brasil = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/produção_por_estado/").glob("*.csv"))

In [ ]:
df = pd.read_csv(nordeste_files[1])
df

,Leite | LEITE AO PRODUTOR CEPEA/ESALQ (R$/litro) - líquido - Bahia,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Nota,,NaN,NaN,NaN,NaN,NaN
1,Fonte,Cepea,NaN,NaN,NaN,NaN,NaN
2,Data,Preço bruto mínimo,Preço bruto médio,Preço bruto máximo,Preço líquido mínimo,Preço líquido médio,Preço líquido máximo
3,12/2004,"0,3927","0,4764","0,5117","0,3663","0,4481","0,4827"
4,01/2005,"0,3790","0,4656","0,5057","0,3459","0,4305","0,4697"
...,...,...,...,...,...,...,...
227,08/2023,-,-,"0,0000","0,0000","2,0923","0,0000"
228,09/2023,-,-,"0,0000","0,0000","2,0860","0,0000"
229,10/2023,-,-,"0,0000","0,0000","2,0785","0,0000"
230,11/2023,-,-,"0,0000","0,0000","2,0601","0,0000"


Leite ao produtor, é o preço de circulação do leite entre os produtores rurais, esse fator ajuda a entender qual o preço do litro do leite que está circulando, ajudando a entender qual o preço do leite para os produtores de derivados do leite.

In [ ]:
df_centro_oeste_processed_all = pd.DataFrame()

all_states_data = []

for file in centro_oeste_files:
  df_centro_oeste = pd.read_csv(file)
  df_centro_oeste = df_centro_oeste.rename(columns={"Unnamed: 1": "Preço bruto mínimo", "Unnamed: 2" : "Preço bruto médio",       "Unnamed: 3": "Preço bruto máximo",     "Unnamed: 4": "Preço líquido mínimo", "Unnamed: 5": "Preço líquido médio", "Unnamed: 6": "Preço líquido máximo"})
  df_centro_oeste.dropna()
  df_centro_oeste = df_centro_oeste.drop(index=0)
  df_centro_oeste = df_centro_oeste.drop(index=1)
  df_centro_oeste = df_centro_oeste.drop(index=2)
  df_centro_oeste = df_centro_oeste.replace('-', 0)
  price_columns = ["Preço bruto mínimo", "Preço bruto médio", "Preço bruto máximo",
                   "Preço líquido mínimo", "Preço líquido médio", "Preço líquido máximo"]
  for col in price_columns:
      df_centro_oeste[col] = df_centro_oeste[col].astype(str).str.replace(',', '.', regex=False).astype(float)

  state_name = df_centro_oeste.columns.values[0][60:80]

  # display(df_centro_oeste)
  df_centro_oeste['Year'] = pd.to_datetime(df_centro_oeste.iloc[:, 0], format='%m/%Y', errors='coerce').dt.year

  # Group df_centro_oeste by year and calculate the mean of 'Preço líquido máximo'
  df_centro_oeste_yearly = df_centro_oeste.groupby('Year')['Preço líquido médio'].mean().reset_index()
  df_centro_oeste_yearly['State'] = state_name

  all_states_data.append(df_centro_oeste_yearly)

df_centro_oeste_processed_all = pd.concat(all_states_data, ignore_index=True)

df_yearly_mean_centro_oeste = df_centro_oeste_processed_all.groupby("Year")['Preço líquido médio'].mean().reset_index()

df_yearly_mean_centro_oeste.rename(columns={'Preço líquido médio': 'Preço líquido médio do leite (entre produtores) centro_oeste'}, inplace=True)

display(df_yearly_mean_centro_oeste)

,Year,Preço líquido médio do leite (entre produtores) centro_oeste
0,2004,0.486200
1,2005,0.482142
2,2006,0.446533
3,2007,0.638908
4,2008,0.655217
5,2009,0.621867
6,2010,0.679992
7,2011,0.799900
8,2012,0.829433
9,2013,0.991642


In [ ]:
df_norte_processed_all = pd.DataFrame()

all_states_data = []

for file in nordeste_files:
  df_nordeste = pd.read_csv(file)
  df_nordeste = df_nordeste.rename(columns={"Unnamed: 1": "Preço bruto mínimo", "Unnamed: 2" : "Preço bruto médio",       "Unnamed: 3": "Preço bruto máximo",     "Unnamed: 4": "Preço líquido mínimo", "Unnamed: 5": "Preço líquido médio", "Unnamed: 6": "Preço líquido máximo"})
  df_nordeste.dropna()
  df_nordeste = df_nordeste.drop(index=0)
  df_nordeste = df_nordeste.drop(index=1)
  df_nordeste = df_nordeste.drop(index=2)
  df_nordeste = df_nordeste.replace('-', 0)
  price_columns = ["Preço bruto mínimo", "Preço bruto médio", "Preço bruto máximo",
                   "Preço líquido mínimo", "Preço líquido médio", "Preço líquido máximo"]
  for col in price_columns:
      df_nordeste[col] = df_nordeste[col].astype(str).str.replace(',', '.', regex=False).astype(float)

  state_name = df_nordeste.columns.values[0][60:80]

  # display(df_nordeste)
  df_nordeste['Year'] = pd.to_datetime(df_nordeste.iloc[:, 0], format='%m/%Y', errors='coerce').dt.year

  # Group df_nordeste by year and calculate the mean of 'Preço líquido máximo'
  df_nordeste_yearly = df_nordeste.groupby('Year')['Preço líquido médio'].mean().reset_index()
  df_nordeste_yearly['State'] = state_name

  all_states_data.append(df_nordeste_yearly)

df_nordeste_processed_all = pd.concat(all_states_data, ignore_index=True)

df_yearly_mean_norte = df_nordeste_processed_all.groupby("Year")['Preço líquido médio'].mean().reset_index()

df_yearly_mean_norte.rename(columns={'Preço líquido médio': 'Preço líquido médio do leite (entre produtores) nordeste'}, inplace=True)

display(df_yearly_mean_norte)

,Year,Preço líquido médio do leite (entre produtores) nordeste
0,2004,0.448100
1,2005,0.427217
2,2006,0.386758
3,2007,0.521083
4,2008,0.600042
5,2009,0.596550
6,2010,0.625583
7,2011,0.667808
8,2012,0.798256
9,2013,0.929396


In [ ]:
df_sud_processed_all = pd.DataFrame()

all_states_data = []

for file in sudeste_files:
  df_sud = pd.read_csv(file)
  df_sud = df_sud.rename(columns={"Unnamed: 1": "Preço bruto mínimo", "Unnamed: 2" : "Preço bruto médio",       "Unnamed: 3": "Preço bruto máximo",     "Unnamed: 4": "Preço líquido mínimo", "Unnamed: 5": "Preço líquido médio", "Unnamed: 6": "Preço líquido máximo"})
  df_sud.dropna()
  df_sud = df_sud.drop(index=0)
  df_sud = df_sud.drop(index=1)
  df_sud = df_sud.drop(index=2)
  df_sud = df_sud.replace('-', 0)
  price_columns = ["Preço bruto mínimo", "Preço bruto médio", "Preço bruto máximo",
                   "Preço líquido mínimo", "Preço líquido médio", "Preço líquido máximo"]
  for col in price_columns:
      df_sud[col] = df_sud[col].astype(str).str.replace(',', '.', regex=False).astype(float)

  state_name = df_sud.columns.values[0][60:80]

  # display(df_sud)
  df_sud['Year'] = pd.to_datetime(df_sud.iloc[:, 0], format='%m/%Y', errors='coerce').dt.year

  # Group df_sud by year and calculate the mean of 'Preço líquido máximo'
  df_sud_yearly = df_sud.groupby('Year')['Preço líquido médio'].mean().reset_index()
  df_sud_yearly['State'] = state_name

  all_states_data.append(df_sud_yearly)

df_sud_processed_all = pd.concat(all_states_data, ignore_index=True)

df_yearly_mean_sud = df_sud_processed_all.groupby("Year")['Preço líquido médio'].mean().reset_index()

df_yearly_mean_sud.rename(columns={'Preço líquido médio': 'Preço líquido médio do leite (entre produtores) SUDESTE'}, inplace=True)

display(df_yearly_mean_sud)

In [ ]:
for file in sul_files:
  df_sul = pd.read_csv(file)
  df_sul = df_sul.rename(columns={"Unnamed: 1": "Preço bruto mínimo", "Unnamed: 2" : "Preço bruto médio",       "Unnamed: 3": "Preço bruto máximo",     "Unnamed: 4": "Preço líquido mínimo", "Unnamed: 5": "Preço líquido médio", "Unnamed: 6": "Preço líquido máximo"}) \
    .drop(index=[0, 1, 2]) \
    .dropna() \
    .replace('-', 0)
  price_columns = ["Preço bruto mínimo", "Preço bruto médio", "Preço bruto máximo",
                   "Preço líquido mínimo", "Preço líquido médio", "Preço líquido máximo"]
  for col in price_columns:
      df_sul[col] = df_sul[col].astype(str).str.replace(',', '.', regex=False).astype(float)

  state_name = df_sul.columns.values[0][60:80]

  # display(df_sud)
  df_sul['Year'] = pd.to_datetime(df_sul.iloc[:, 0], format='%m/%Y', errors='coerce').dt.year

  df_sul_yearly = df_sul.groupby('Year')['Preço líquido médio'].mean().reset_index()
  df_sul_yearly['State'] = state_name

  all_states_data.append(df_sul_yearly)

  df_sud_processed_all = pd.concat(all_states_data, ignore_index=True)

  df_yearly_mean_sul = df_sud_processed_all.groupby("Year")['Preço líquido médio'].mean().reset_index()

  df_yearly_mean_sul.rename(columns={'Preço líquido médio': 'Preço líquido médio do leite (entre produtores) SUL'}, inplace=True)

display(df_yearly_mean_sul)

Dado A falta de dados que deriva dos anos 2020 até 2023 das regiões foi utilizados o método KNN pela sua eficácia em problemas de impute em séries históricas.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

df_sud.replace(0, np.nan)

df_with_nan = df_sud.copy()

df_with_nan = df_with_nan.iloc[:, 1:]

scaler = StandardScaler()

columns = df_with_nan.columns
index = df_with_nan.index

df_scaled = pd.DataFrame(scaler.fit_transform(df_with_nan), columns=columns)

imputer = KNNImputer(n_neighbors=10)

df_imputed_scaled = pd.DataFrame(imputer.fit_transform(df_scaled), columns=columns)

df_imputed = pd.DataFrame(scaler.inverse_transform(df_imputed_scaled), columns=columns, index=index)

display(df_imputed)

,Preço bruto mínimo,Preço bruto médio,Preço bruto máximo,Preço líquido mínimo,Preço líquido médio,Preço líquido máximo,Year
3,0.7861,8.607000e-01,0.9315,0.7113,0.7843,0.8535,2012.0
4,0.8083,8.824000e-01,0.9536,0.7364,0.8089,0.8784,2012.0
5,0.7770,8.573000e-01,0.9316,0.7025,0.7814,0.8536,2012.0
6,0.7744,8.533000e-01,0.9230,0.7018,0.7798,0.8455,2012.0
7,0.7726,8.503000e-01,0.9227,0.7021,0.7789,0.8472,2012.0
...,...,...,...,...,...,...,...
157,0.0000,-1.110223e-16,0.0000,0.0000,2.6594,0.0000,2025.0
158,0.0000,-1.110223e-16,0.0000,0.0000,2.7835,0.0000,2025.0
159,0.0000,-1.110223e-16,0.0000,0.0000,2.6314,0.0000,2025.0
160,0.0000,-1.110223e-16,0.0000,0.0000,2.6074,0.0000,2025.0


Produção de leite total nas Regiões do Brasil em milhões de litros (CIleite)

In [ ]:
producao = pd.read_csv(produtividade_brasil[0])
producao

Relação produtividade de leite e o preço líquido do leite

In [ ]:
# Extract year from the first column of df_sud
df_sud['Year'] = pd.to_datetime(df_sud.iloc[:, 0], format='%m/%Y', errors='coerce').dt.year

# Group df_sud by year and calculate the mean of 'Preço líquido máximo'
df_sud_yearly = df_sud.groupby('Year')['Preço líquido médio'].mean().reset_index()

df_sud_yearly


###Obter dados de inflação

In [ ]:
inflacao_df = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/inflação/").glob("*.csv"))

In [ ]:
inflacao_data = pd.read_csv(inflacao_df[0])
coluns_filter = ['ano', 'mes', 'ipca_variacao']
inflacao_df = inflacao_data[coluns_filter]
media_ipca_ano = inflacao_df.groupby('ano')['ipca_variacao'].mean().reset_index()
media_ipca_ano
media_ipca_df = pd.DataFrame(media_ipca_ano['ipca_variacao'], index=range(20,44))
media_ipca_df

#TODO: É preciso que eu solicite novamente no inmet os dados de temperatura e precipatação do sul e do sudeste, porém dessa vez tenho que fazer separadamente para cada região. Para facilitar o processamento

Depois disso eu ploto quais as tendências de aumento de produção entre sul e sudeste

Fazer uma relação entre dados econômicos como inflação e o preço do dolar afetam as produções e como cada região reage a tais influencias, verificar qual região sofre mais com tais mudanças

Fazer a mesma coisa com dados climáticos entre esses estados

Se sobrar tempo, utilizar random florest para prever o preço do leite com base nesses fatores

In [ ]:
tempPrec_sudeste = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/temperature_precipitacao/tempPrec_sudeste/").glob("*.csv"))
tempPrec_sul = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/temperature_precipitacao/tempPrec_sul/").glob("*.csv"))
tempPrec_centro_oeste = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/temperature_precipitacao/tempPrec_centro_oeste/").glob("*.csv"))
tempPrec_nordeste = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/temperature_precipitacao/tempPrec_nordeste/").glob("*.csv"))
tempPrec_norte = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/temperature_precipitacao/tempPrec_norte/").glob("*.csv"))

In [ ]:
name = pd.read_csv(tempPrec_sudeste[9], skiprows=10, sep=';')
name

,Data Medicao,"PRECIPITACAO TOTAL, MENSAL (AUT)(mm)","TEMPERATURA MEDIA, MENSAL (AUT)(°C)",Unnamed: 3
0,2008-06-30,NaN,NaN,NaN
1,2008-07-31,18.0,20.5,NaN
2,2008-08-31,38.0,22.2,NaN
3,2008-09-30,93.0,21.5,NaN
4,2008-10-31,169.4,23.4,NaN
...,...,...,...,...
195,2024-09-30,51.6,23.9,NaN
196,2024-10-31,241.4,23.9,NaN
197,2024-11-30,130.8,24.4,NaN
198,2024-12-31,227.6,25.7,NaN


# Capturar os dados de preciptação e temperatura da região Sul e Sudeste separadamente

Ideia: para todos os valores de todos os arquivos de precipitação e temperatura eu posso pegar todos os valores relacionados a uma mesma data, coloca-los em uma lista e depois soma-los



In [ ]:
import warnings
warnings.filterwarnings("ignore")

array_year = []
dict_temp_sul = {}
dict_prec_sul = {}

for file in tempPrec_sul:
  array_temp = []
  array_prec = []
  year_date = ["2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
  df_prectemp_notfilter = pd.read_csv(file, skiprows=10, sep=';')
  tempPrec_sul_df = df_prectemp_notfilter.drop(columns="Unnamed: 3")

  for date in year_date:
    for index, row in tempPrec_sul_df.iterrows():
      if date in row['Data Medicao']:
        array_temp.append(row[2])
        array_prec.append(row[1])

    if array_temp:
      median_by_year = np.nanmean(array_temp)
      dict_temp_sul.update({f"{file.name[6:11]}_{date}": median_by_year})
      dict_prec_sul.update({f"{file.name[6:11]}_{date}": st.median(array_prec)})
    else:
      dict_temp_sul.update({f"{file.name[6:11]}": None})


In [ ]:
import warnings
warnings.filterwarnings("ignore")

array_year = []
dict_temp_sudeste = {}
dict_prec_sudeste = {}

for file in tempPrec_sudeste:
  array_temp = []
  array_prec = []
  year_date = ["2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
  df_prectemp_notfilter = pd.read_csv(file, skiprows=10, sep=';')
  tempPrec_sudeste_df = df_prectemp_notfilter.drop(columns="Unnamed: 3")


  for date in year_date:
    for index, row in tempPrec_sudeste_df.iterrows():
      if date in str(row['Data Medicao']):
        array_temp.append(row[2])
        array_prec.append(row[1])

    if array_temp:
      array_temp = [str(n).replace(",", ".") for n in array_temp]
      array_temp = [float(n) if n != 'nan' else np.nan for n in array_temp]
      array_prec = [str(n).replace(",", ".") for n in array_prec]
      array_prec = [float(n) if n != 'nan' else np.nan for n in array_prec]


      median_by_year = np.nanmean(array_temp)
      dict_temp_sudeste.update({f"{file.name[6:11]}_{date}": median_by_year})
      dict_prec_sudeste.update({f"{file.name[6:11]}_{date}": np.nanmedian(array_prec)})
    else:
      dict_temp_sudeste.update({f"{file.name[6:11]}": None})

In [ ]:
import warnings
warnings.filterwarnings("ignore")

array_year = []
dict_temp_centro_oeste = {}
dict_prec_centro_oeste = {}

for file in tempPrec_centro_oeste:
  array_temp = []
  array_prec = []
  year_date = ["2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
  df_prectemp_notfilter = pd.read_csv(file, skiprows=10, sep=';')
  tempPrec_centro_oeste_df = df_prectemp_notfilter.drop(columns="Unnamed: 3")

  for date in year_date:
    for index, row in tempPrec_centro_oeste_df.iterrows():
      if date in str(row['Data Medicao']):
        array_temp.append(row[2])
        array_prec.append(row[1])

    if array_temp:
      array_temp = [str(n).replace(",", ".") for n in array_temp] 
      array_temp = [float(n) if n != 'nan' else np.nan for n in array_temp] 
      array_prec = [str(n).replace(",", ".") for n in array_prec]
      array_prec = [float(n) if n != 'nan' else np.nan for n in array_prec]

      median_by_year = np.nanmean(array_temp)
      dict_temp_centro_oeste.update({f"{file.name[6:11]}_{date}": median_by_year})
      dict_prec_centro_oeste.update({f"{file.name[6:11]}_{date}": np.nanmedian(array_prec)})
    else:
      dict_temp_centro_oeste.update({f"{file.name[6:11]}": None})

In [ ]:
print(type(array_temp[1]))

<class 'float'>


In [ ]:
import warnings
warnings.filterwarnings("ignore")

array_year = []
dict_temp_norte = {}
dict_prec_norte = {}

for file in tempPrec_norte:
  array_temp = []
  array_prec = []
  year_date = ["2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
  df_prectemp_notfilter = pd.read_csv(file, skiprows=10, sep=';')
  tempPrec_norte_df = df_prectemp_notfilter.drop(columns="Unnamed: 3")

  for date in year_date:
    for index, row in tempPrec_norte_df.iterrows():
      if date in str(row['Data Medicao']):
        array_temp.append(row[2])
        array_prec.append(row[1])

    if array_temp:
      array_temp = [str(n).replace(",", ".") for n in array_temp]
      array_temp = [float(n) if n != 'nan' else np.nan for n in array_temp]
      array_prec = [str(n).replace(",", ".") for n in array_prec]
      array_prec = [float(n) if n != 'nan' else np.nan for n in array_prec]

      median_by_year = np.nanmean(array_temp)
      dict_temp_norte.update({f"{file.name[6:11]}_{date}": median_by_year})
      dict_prec_norte.update({f"{file.name[6:11]}_{date}": np.nanmedian(array_prec)})
    else:
      dict_temp_norte.update({f"{file.name[6:11]}": None})


In [ ]:
import warnings
warnings.filterwarnings("ignore")

array_year = []
dict_temp_nordeste = {}
dict_prec_nordeste = {}

for file in tempPrec_nordeste:
  array_temp = []
  array_prec = []
  year_date = ["2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
  df_prectemp_notfilter = pd.read_csv(file, skiprows=10, sep=';')
  tempPrec_nordeste_df = df_prectemp_notfilter.drop(columns="Unnamed: 3")


  for date in year_date:
    for index, row in tempPrec_nordeste_df.iterrows():
      if date in str(row['Data Medicao']):
        array_temp.append(row[2])
        array_prec.append(row[1])

    if array_temp:
      array_temp = [str(n).replace(",", ".") for n in array_temp]
      array_temp = [float(n) if n != 'nan' else np.nan for n in array_temp]
      array_prec = [str(n).replace(",", ".") for n in array_prec]
      array_prec = [float(n) if n != 'nan' else np.nan for n in array_prec]

      median_by_year = np.nanmean(array_temp)
      dict_temp_nordeste.update({f"{file.name[6:11]}_{date}": median_by_year})
      dict_prec_nordeste.update({f"{file.name[6:11]}_{date}": np.nanmedian(array_prec)})
    else:
      dict_temp_nordeste.update({f"{file.name[6:11]}": None})

Plot the temp and the preciptation over the years

## Dataframe de temperaturas e precipitação do sul

In [ ]:
years_temp = [key.split('_')[-1] for key in dict_temp_sudeste.keys() if dict_temp_sudeste[key] is not None and '_' in key]
temperatures = [value for value in dict_temp_sudeste.values() if value is not None]

years_prec = [key.split('_')[-1] for key in dict_prec_sul.keys() if dict_prec_sul[key] is not None and '_' in key]
preciptation = [value for value in dict_prec_sul.values() if value is not None]

print(years_temp)
print(temperatures)

# CRIA O DATAFRAME DE TEMPERATURAS DO SUL
temp_sul_df = pd.DataFrame({'Year': years_temp, 'Temperature': temperatures})
temp_sul_df['Year'] = temp_sul_df['Year'].astype(int)

#CRIA O DATAFRAME DE PRECIPTAÇÃO DO SUL
prec_sul_df = pd.DataFrame({'Year': years_prec, 'Preciptation': preciptation})
prec_sul_df['Year'] = prec_sul_df['Year'].astype(int)

temp_sul_df
# prec_sul_df

['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',

,Year,Temperature
0,2011,22.266667
1,2012,22.866667
2,2013,22.381481
3,2014,22.248718
4,2015,22.425490
...,...,...
2444,2019,23.709701
2445,2020,23.682517
2446,2021,23.709804
2447,2022,23.768553


In [ ]:
years_temp = [key.split('_')[-1] for key in dict_temp_centro_oeste.keys() if dict_temp_centro_oeste[key] is not None and '_' in key]
temperatures = [value for value in dict_temp_centro_oeste.values() if value is not None]

years_prec = [key.split('_')[-1] for key in dict_prec_centro_oeste.keys() if dict_prec_centro_oeste[key] is not None and '_' in key]
preciptation = [value for value in dict_prec_centro_oeste.values() if value is not None]

print(years_temp)
print(temperatures)

temp_centro_oeste_df = pd.DataFrame({'Year': years_temp, 'Temperature': temperatures})
temp_centro_oeste_df['Year'] = temp_centro_oeste_df['Year'].astype(int)

prec_centro_oeste_df = pd.DataFrame({'Year': years_prec, 'Preciptation': preciptation})
prec_centro_oeste_df['Year'] = prec_centro_oeste_df['Year'].astype(int)

display(temp_centro_oeste_df)
display(prec_centro_oeste_df)

['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2007', '2008', '2009', '2010', '2011', '2012', '2013',

,Year,Temperature
0,2008,22.200000
1,2009,21.200000
2,2010,21.229630
3,2011,21.102564
4,2012,21.137255
...,...,...
1685,2019,25.224000
1686,2020,25.296350
1687,2021,25.296350
1688,2022,25.262069


,Year,Preciptation
0,2008,205.2
1,2009,169.1
2,2010,138.6
3,2011,138.6
4,2012,115.8
...,...,...
1685,2019,64.2
1686,2020,62.4
1687,2021,62.4
1688,2022,66.0


In [ ]:
years_temp = [key.split('_')[-1] for key in dict_temp_norte.keys() if dict_temp_norte[key] is not None and '_' in key]
temperatures = [value for value in dict_temp_norte.values() if value is not None]

years_prec = [key.split('_')[-1] for key in dict_prec_norte.keys() if dict_prec_norte[key] is not None and '_' in key]
preciptation = [value for value in dict_prec_norte.values() if value is not None]

temp_norte_df = pd.DataFrame({'Year': years_temp, 'Temperature': temperatures})
temp_norte_df['Year'] = temp_norte_df['Year'].astype(int)

prec_norte_df = pd.DataFrame({'Year': years_prec, 'Preciptation': preciptation})
prec_norte_df['Year'] = prec_norte_df['Year'].astype(int)

display(temp_norte_df)
display(prec_norte_df)

,Year,Temperature
0,2015,25.937500
1,2016,25.860000
2,2017,25.775000
3,2018,25.600000
4,2019,25.557143
...,...,...
1258,2019,26.442553
1259,2020,26.357627
1260,2021,26.223881
1261,2022,26.176316


,Year,Preciptation
0,2015,90.6
1,2016,130.6
2,2017,165.8
3,2018,176.0
4,2019,196.5
...,...,...
1258,2019,53.4
1259,2020,60.3
1260,2021,43.6
1261,2022,50.4


In [ ]:
years_temp = [key.split('_')[-1] for key in dict_temp_nordeste.keys() if dict_temp_nordeste[key] is not None and '_' in key]
temperatures = [value for value in dict_temp_nordeste.values() if value is not None]

years_prec = [key.split('_')[-1] for key in dict_prec_nordeste.keys() if dict_prec_nordeste[key] is not None and '_' in key]
preciptation = [value for value in dict_prec_nordeste.values() if value is not None]

temp_nordeste_df = pd.DataFrame({'Year': years_temp, 'Temperature': temperatures})
temp_nordeste_df['Year'] = temp_nordeste_df['Year'].astype(int)

prec_nordeste_df = pd.DataFrame({'Year': years_prec, 'Preciptation': preciptation})
prec_nordeste_df['Year'] = prec_nordeste_df['Year'].astype(int)

display(temp_nordeste_df)
display(prec_nordeste_df)

,Year,Temperature
0,2008,23.575000
1,2009,24.345000
2,2010,24.425000
3,2011,24.370455
4,2012,24.332143
...,...,...
2368,2019,24.271318
2369,2020,24.230435
2370,2021,24.224667
2371,2022,24.231818


,Year,Preciptation
0,2008,65.4
1,2009,63.0
2,2010,76.0
3,2011,76.0
4,2012,71.0
...,...,...
2368,2019,39.6
2369,2020,36.6
2370,2021,36.2
2371,2022,36.6


In [ ]:
years_temp = [key.split('_')[-1] for key in dict_temp_sul.keys() if dict_temp_sul[key] is not None and '_' in key]
temperatures = [value for value in dict_temp_sul.values() if value is not None]

years_prec = [key.split('_')[-1] for key in dict_prec_sudeste.keys() if dict_prec_sudeste[key] is not None and '_' in key]
preciptation = [value for value in dict_prec_sudeste.values() if value is not None]

print(years_temp)
print(temperatures)

temp_sudeste_df = pd.DataFrame({'Year': years_temp, 'Temperature': temperatures})
temp_sudeste_df['Year'] = temp_sul_df['Year'].astype(int)

prec_sudeste_df = pd.DataFrame({'Year': years_prec, 'Preciptation': preciptation})
prec_sudeste_df['Year'] = prec_sudeste_df['Year'].astype(int)

prec_sudeste_df
temp_sudeste_df

['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2008', '2009', '2010', '2011', '2012',

,Year,Temperature
0,2011,16.883333
1,2012,17.500000
2,2013,17.230000
3,2014,17.316667
4,2015,17.287037
...,...,...
1373,2021,18.414925
1374,2022,18.468493
1375,2023,18.465605
1376,2006,18.520000




--- --- ---

---








tendência de aumento de produtividade

Interessante visualizar que mesmo que a tendencia de aumento de produção de leite entre a região sul e sudeste não sejam tão fortes, ainda assim o brasil está com uma boa tendência de aumento na produção do leite, isso se deve ao fato de outras regiões, principalmente a do nordeste está com alta tendência de aumento de produção de leite

#Obtendo dados do preço da produção de leite para o produtor rural

In [ ]:
data_prod_preco = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/Preco_do_leite_ao_produtor/").glob("*.csv"))

In [ ]:
df_prod_preco = pd.read_csv(data_prod_preco[0])

df_prod_preco['Year'] = df_prod_preco['Unnamed: 0'].astype(int)
df_prod_preco.drop(columns=['Unnamed: 0'], inplace=True)

state_columns = ["São Paulo", "Minas Gerais", "Rio Grande do Sul", "Paraná", "Santa Catarina", "Goiás", "Bahia", "BRASIL"]

for col in state_columns:
    if col in df_prod_preco.columns:
        df_prod_preco[col] = df_prod_preco[col].astype(str).str.replace(',', '.', regex=False)
        df_prod_preco[col] = pd.to_numeric(df_prod_preco[col], errors='coerce')


df_prod_preco_sul = df_prod_preco[["Year", "Rio Grande do Sul", "Paraná", "Santa Catarina"]]
df_prod_preco_sudeste = df_prod_preco[["Year", "São Paulo", "Minas Gerais"]]
df_prod_preco_centro_oeste = df_prod_preco[["Year", "Goiás"]]
df_prod_preco_nordeste = df_prod_preco[["Year", "Bahia"]]

df_prod_preco_sul_mean = df_prod_preco_sul.set_index('Year').mean(axis=1).to_frame(name='média do preço do leite para o produtor no Sul').reset_index()
df_prod_preco_sudeste_mean = df_prod_preco_sudeste.set_index('Year').mean(axis=1).to_frame(name='média do preço do leite para o produtor no Sudeste').reset_index()

df_prod_preco_centro_oeste_mean = df_prod_preco_centro_oeste.set_index('Year').reset_index()
df_prod_preco_nordeste_mean = df_prod_preco_nordeste.set_index('Year').reset_index()

df_prod_preco_norte_mean.rename(columns={'Bahia': 'média do preço do leite para o produtor no Nordeste'}, inplace=True)
df_prod_preco_centro_oeste_mean.rename(columns={'Goiás': 'média do preço do leite para o produtor no Centro-Oeste'}, inplace=True)

display(df_prod_preco_centro_oeste_mean)

,Year,média do preço do leite para o produtor no Centro-Oeste
0,2007,2.66
1,2008,2.48
2,2009,2.30
3,2010,2.46
4,2011,2.44
5,2012,2.13
6,2013,2.33
7,2014,2.15
8,2015,1.95
9,2016,2.22


# Dados de número de estações da embrapa em cada região

In [ ]:
embrapa_unit = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/unidades_embrapa/").glob("*.csv"))
df_embrapa = pd.read_csv(embrapa_unit[0])

embrapa_regions_units = df_embrapa.groupby("Regiao").size().reset_index(name="quantidade_de_unidades")
display(embrapa_regions_units)

,Regiao,quantidade_de_unidades
0,Centro-Oeste,10
1,Nordeste,9
2,Norte,7
3,Sudeste,10
4,Sul,7


# Dados número de universidades relevantes para produção na área do agro

https://www.usnews.com/education/best-global-universities/brazil/agricultural-sciences

In [ ]:
universits = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/universidades/").glob("*.csv"))
df_universits = pd.read_csv(universits[0])
df_universits

,regions of brazil,agregate number
0,Sudeste,12
1,Sul,12
2,Nordeste,8
3,Centro-Oeste,3
4,Norte,1


# COnstrução do DATASET FINAL

NO final eu tenho um dataset com Produção de leite (sul, sudeste), variação do IPCA, temperatura e preciptação(sul e sudeste) e preço de produção do leite deflação, consumo de leite per capita

In [ ]:
dataset_geral = pd.DataFrame()
dataset_geral['year'] = range(2000, 2024)
dataset_geral["média da produção de leite sudeste (milhões)"] = producao["Sudeste"]
dataset_geral["média da produção de leite sul (milhões)"] = producao["Sul"]
dataset_geral["média de produção de leite no centro-oeste"] = producao["Centro-Oeste"]
dataset_geral["média de produção de leite no norte"] = producao["Norte"]
dataset_geral["média de produção de leite no nordeste"] = producao["Nordeste"]

media_ipca_ano_renamed = media_ipca_ano.rename(columns={'ano': 'year'})
dataset_geral = pd.merge(dataset_geral, media_ipca_ano_renamed, on='year', how='left')

temp_sul_yearly_median = temp_sul_df.groupby('Year')['Temperature'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Temperature': 'Temperatura média SUL (°C)'})
dataset_geral = pd.merge(dataset_geral, temp_sul_yearly_median, on='year', how='left')

prec_sul_yearly_median = prec_sul_df.groupby('Year')['Preciptation'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Preciptation': 'Precipitação média SUL (mm)'})
dataset_geral = pd.merge(dataset_geral, prec_sul_yearly_median, on='year', how='left')

temp_sudeste_yearly_median = temp_sudeste_df.groupby('Year')['Temperature'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Temperature': 'Temperatura média SUDESTE (°C)'})
dataset_geral = pd.merge(dataset_geral, temp_sudeste_yearly_median, on='year', how='left')

prec_sudeste_yearly_median = prec_sudeste_df.groupby('Year')['Preciptation'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Preciptation': 'Precipitação média SUDESTE (mm)'})
dataset_geral = pd.merge(dataset_geral, prec_sudeste_yearly_median, on='year', how='left')

df_prod_preco_sul_mean_renamed = df_prod_preco_sul_mean.rename(columns={'Year': 'year'})
dataset_geral = pd.merge(dataset_geral, df_prod_preco_sul_mean_renamed, on='year', how='left')

df_prod_preco_sudeste_mean_renamed = df_prod_preco_sudeste_mean.rename(columns={'Year': 'year'})
dataset_geral = pd.merge(dataset_geral, df_prod_preco_sudeste_mean_renamed, on='year', how='left')

df_prod_preco_centro_oeste_mean = df_prod_preco_centro_oeste_mean.rename(columns={'Year': 'year'})
dataset_geral = pd.merge(dataset_geral, df_prod_preco_centro_oeste_mean, on='year', how='left')

df_prod_preco_norte_mean = df_prod_preco_norte_mean.rename(columns={'Year': 'year'})
dataset_geral = pd.merge(dataset_geral, df_prod_preco_norte_mean, on='year', how='left')

df_yearly_mean_sud_renamed = df_yearly_mean_sud.rename(columns={'Year': 'year'})
dataset_geral = pd.merge(dataset_geral, df_yearly_mean_sud_renamed, on='year', how='left')

df_yearly_mean_sul_renamed = df_yearly_mean_sul.rename(columns={'Year': 'year'})
dataset_geral = pd.merge(dataset_geral, df_yearly_mean_sul_renamed, on='year', how='left')

df_yearly_mean_centro_oeste_renamed = df_yearly_mean_centro_oeste.rename(columns={'Year': 'year'})
dataset_geral = pd.merge(dataset_geral, df_yearly_mean_centro_oeste_renamed, on='year', how='left')

df_yearly_mean_norte_renamed = df_yearly_mean_norte.rename(columns={'Year': 'year'})
dataset_geral = pd.merge(dataset_geral, df_yearly_mean_norte_renamed, on='year', how='left')


dataset_filtered = dataset_geral.copy()

display(dataset_filtered)

NameError: name 'df_yearly_mean_centro_oeste' is not defined

In [ ]:
temp_centro_oeste_yearly_median = temp_centro_oeste_df.groupby('Year')['Temperature'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Temperature': 'Temperatura média CENTRO-OESTE (°C)'})
dataset_geral = pd.merge(dataset_geral, temp_centro_oeste_yearly_median, on='year', how='left')

prec_centro_oeste_yearly_median = prec_centro_oeste_df.groupby('Year')['Preciptation'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Preciptation': 'Precipitação média CENTRO-OESTE (mm)'})
dataset_geral = pd.merge(dataset_geral, prec_centro_oeste_yearly_median, on='year', how='left')

temp_norte_yearly_median = temp_norte_df.groupby('Year')['Temperature'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Temperature': 'Temperatura média NORTE (°C)'})
dataset_geral = pd.merge(dataset_geral, temp_norte_yearly_median, on='year', how='left')

prec_norte_yearly_median = prec_norte_df.groupby('Year')['Preciptation'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Preciptation': 'Precipitação média NORTE (mm)'})
dataset_geral = pd.merge(dataset_geral, prec_norte_yearly_median, on='year', how='left')

temp_nordeste_yearly_median = temp_nordeste_df.groupby('Year')['Temperature'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Temperature': 'Temperatura média NORDESTE (°C)'})
dataset_geral = pd.merge(dataset_geral, temp_nordeste_yearly_median, on='year', how='left')

prec_nordeste_yearly_median = prec_nordeste_df.groupby('Year')['Preciptation'].agg(lambda x: np.nanmedian(x)).reset_index().rename(columns={'Year': 'year', 'Preciptation': 'Precipitação média NORDESTE (mm)'})
dataset_geral = pd.merge(dataset_geral, prec_nordeste_yearly_median, on='year', how='left')

dataset_filtered = dataset_geral.copy()
display(dataset_filtered)

,year,média da produção de leite sudeste (milhões),média da produção de leite sul (milhões),média de produção de leite no centro-oeste,média de produção de leite no norte,média de produção de leite no nordeste,ipca_variacao,Temperatura média SUL (°C),Precipitação média SUL (mm),Temperatura média SUDESTE (°C),...,média do preço do leite para o produtor no Centro-Oeste,média do preço do leite para o produtor no Norte,Preço líquido médio do leite (entre produtores) SUDESTE,Preço líquido médio do leite (entre produtores) SUL,Temperatura média CENTRO-OESTE (°C),Precipitação média CENTRO-OESTE (mm),Temperatura média NORTE (°C),Precipitação média NORTE (mm),Temperatura média NORDESTE (°C),Precipitação média NORDESTE (mm)
0,2000,8.574,4.904,3.080,1.050,2.159,0.485833,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,8.573,5.188,3.246,1.237,2.266,0.618333,23.417045,NaN,NaN,...,NaN,NaN,NaN,NaN,23.825000,108.00,26.975000,147.00,25.433333,167.30
2,2002,8.746,5.508,3.460,1.567,2.363,0.991667,24.550000,251.90,18.640625,...,NaN,NaN,NaN,NaN,24.468750,89.00,26.907143,147.00,25.067081,22.80
3,2003,8.934,5.779,3.535,1.498,2.508,0.745833,22.975000,156.60,20.035455,...,NaN,NaN,NaN,NaN,24.396786,86.65,26.371429,164.90,25.467647,55.70
4,2004,9.241,6.246,3.620,1.663,2.705,0.612500,23.070833,115.10,18.863525,...,NaN,NaN,0.502400,0.491250,24.496023,76.15,26.230769,164.90,25.754545,63.80
5,2005,9.535,6.592,3.778,1.743,2.972,0.462500,22.333333,135.30,17.812821,...,NaN,NaN,0.486996,0.455405,24.508824,65.80,26.428571,131.00,25.982222,63.60
6,2006,9.740,7.039,3.722,1.699,3.198,0.258333,22.400000,86.90,18.823256,...,NaN,NaN,0.470121,0.436067,25.094444,130.20,26.472571,131.60,26.043333,57.30
7,2007,9.803,7.510,3.808,1.677,3.339,0.364167,22.600000,115.20,19.102222,...,2.66,2.17,0.619447,0.585231,24.641176,76.15,26.320879,81.20,26.380328,21.00
8,2008,10.141,8.268,4.055,1.666,3.455,0.479167,22.090909,110.80,19.120767,...,2.48,2.27,0.650278,0.620133,24.518644,76.75,26.405263,103.75,25.830000,25.90
9,2009,10.420,8.957,4.222,1.673,3.813,0.352500,22.202945,125.75,19.066960,...,2.30,2.21,0.635711,0.624733,24.454321,86.50,26.360000,141.00,26.000000,41.90


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

df_with_nan = dataset_filtered.copy()

scaler = StandardScaler()
columns = df_with_nan.columns
index = df_with_nan.index

df_scaled = pd.DataFrame(scaler.fit_transform(df_with_nan), columns=columns)

imputer = KNNImputer(n_neighbors=5)

df_imputed_scaled = pd.DataFrame(imputer.fit_transform(df_scaled), columns=columns)

df_imputed = pd.DataFrame(scaler.inverse_transform(df_imputed_scaled), columns=columns, index=index)

display(df_imputed)

## Dowload dataset

In [ ]:
df_imputed.to_csv('dataset_final.csv', index=False)

In [ ]:
!pip install -q pydot
!pip install rdflib
!pip install pydot
!pip install graphviz
!pip install pyvis
!pip install prov

# Aqui eu desenvolvo a proveniencia dos dados utilizados

In [ ]:
import prov
import datetime

In [ ]:
from prov.model import ProvDocument

In [ ]:
document = prov.model.ProvDocument()

document.set_default_namespace('http://example.org/')

e1 = document.entity('test.pdf', {prov.constants.PROV_ENTITY: 'File'})

a1 = document.activity('a1', datetime.datetime.now(), None, {prov.constants.PROV_ACTIVITY: 'script execution'})

document.wasGeneratedBy(e1, a1, None, {'ex:script': 'script.py'})

document.get_provn()

'document\n  default <http://example.org/>\n  \n  entity(test.pdf, [prov:Entity="File"])\n  activity(a1, 2025-11-18T18:15:02.298339, -, [prov:Activity="script execution"])\n  wasGeneratedBy(test.pdf, a1, -)\nendDocument'

## Dataset de precipitação e temperatura

In [ ]:
doc = ProvDocument()
doc.add_namespace("project_of_dataset_creation", "max_felipe")

doc.add_namespace('data', 'http://example.org/mydata#')
doc.add_namespace('project_script', 'http://example.org/myscript#')
doc.add_namespace('project_of_dataset_creation', 'http://example.org/myproject#')
doc.add_namespace("climatics_sul_region_data", "https://bdmep.inmet.gov.br/")
doc.add_namespace("climatics_sudeste_region_data", "https://bdmep.inmet.gov.br/")

data_climatics_inmet = doc.entity("data:website_climatics_data_inmet", {
    "prov:label": "csv_region_data",
    "prov:Location": "https://bdmep.inmet.gov.br/",
    "prov:description": "csv data of preciptation and temperature for each region in Brasil"
})

data_sul_prov = doc.entity("data:climatics_data_region_csv", {
})

data_sudeste_prov = doc.entity("data:climatics_data_region_csv", {
    "prov:label": "csv data of preciptation and temperature in a region of Brasil",
})

data_Dataframe_sul = doc.entity("data:dataframe_in_memory", {
    "prov:label": "dataframe_data",
    "prov:description": "Dataframe of a region generated by pandas storage in memory"
})

data_Dataframe_sudeste = doc.entity("data:dataframe_in_memory", {
    "prov:label": "dataframe_data",
    "prov:description": "Dataframe of a region generated by pandas storage in memory"
})

download_activity_climatics = doc.activity('data:download_and_save', other_attributes={
    'prov:label': 'Download and Save as CSV'
})

main_process = doc.activity("project_script:script_de_processamento_leite.ipynb", other_attributes= {'prov:label': 'data_manipulation_main_script'})

doc.used(main_process, data_sudeste_prov)
doc.wasGeneratedBy(data_Dataframe_sul, main_process)
doc.wasGeneratedBy(data_Dataframe_sudeste, main_process)
doc.used(download_activity_climatics, data_climatics_inmet)
doc.wasGeneratedBy(data_sul_prov, download_activity_climatics)
doc.wasGeneratedBy(data_sudeste_prov, download_activity_climatics)

print(doc.get_provn())

In [ ]:
from prov.dot import prov_to_dot
dot = prov_to_dot(doc)
dot.write_png('prov_climatics_data.png')

## Dataset de produtividade

In [ ]:
from prov.model import ProvDocument

doc = ProvDocument()
doc.add_namespace('data', 'http://example.org/mydata#')
doc.add_namespace('project', 'http://example.org/myproject#')
doc.add_namespace('inmet', 'https://bdmep.inmet.gov.br/#')

website_data = doc.entity('data:website_milk_production', {
    'prov:label': 'Milk Production in Millions (Website)',
    'prov:location': 'https://www.cileite.com.br/leite_numeros_producao'
})

csv_file = doc.entity('data:milk_production_csv', {
    'prov:label': 'CSV Data of Milk Production per region in Brasil',
})

dataframe_in_memory = doc.entity('data:dataframe_production', {
    'prov:label': 'DataFrame of Milk Production per region in Brasil',
    'prov:description': 'Dataframe generated by pandas storage in memory'
})

download_activity = doc.activity('project:download_and_save', other_attributes={
    'prov:label': 'Download and Save as CSV'
})

main_process_activity = doc.activity('project:script_de_processamento_leite.ipynb', other_attributes= {
    'prov:label': 'Data Manipulation Main Script'
})

doc.used(download_activity, website_data)
doc.wasGeneratedBy(csv_file, download_activity)

doc.used(main_process_activity, csv_file)
doc.wasGeneratedBy(dataframe_in_memory, main_process_activity)

print(doc.get_provn())

In [ ]:
from prov.dot import prov_to_dot
dot = prov_to_dot(doc)
dot.write_png('prov_production_data.png')

## Dataset Média do preço do leite para o produtor


In [ ]:
from prov.model import ProvDocument

doc = ProvDocument()
doc.add_namespace('data', 'http://example.org/mydata#')
doc.add_namespace('project', 'http://example.org/myproject#')
doc.add_namespace('inmet', 'https://bdmep.inmet.gov.br/#')

website_data = doc.entity('data:website_milk_price_to_production', {
    'prov:label': 'Milk price to productor in (Website)',
    'prov:location': 'https://www.cileite.com.br/leite_numeros_precos'
})

csv_file = doc.entity('data:milk_price_to_production_csv', {
    'prov:label': 'CSV Data of Milk price to production for each region',
})

dataframe_in_memory = doc.entity('data:dataframe_price_production', {
    'prov:label': 'DataFrame of Milk price to production',
    'prov:description': 'Dataframe generated by pandas storage in memory'
})

download_activity = doc.activity('project:download_and_save', other_attributes={
    'prov:label': 'Download and Save as CSV'
})

main_process_activity = doc.activity('project:script_de_processamento_leite.ipynb', other_attributes= {
    'prov:label': 'Data Manipulation Main Script'
})

doc.used(download_activity, website_data)
doc.wasGeneratedBy(csv_file, download_activity)

doc.used(main_process_activity, csv_file)
doc.wasGeneratedBy(dataframe_in_memory, main_process_activity)

print(doc.get_provn())

In [ ]:
from prov.dot import prov_to_dot
dot = prov_to_dot(doc)
dot.write_png('prov_price_production_data.png')

# Preço do leite em circulação entre produtores

In [ ]:
doc = ProvDocument()
doc.add_namespace('data', 'http://example.org/mydata#')
doc.add_namespace('project', 'http://example.org/myproject#')
doc.add_namespace('cepea', 'https://www.cepea.org.br/br/indicador/leite.aspx')


website_data = doc.entity('data:milk_price_beetwen_producers_brasil', {
    'prov:label': 'milk price beetwen producers in brasil;',
    'prov:location': 'https://www.cepea.org.br/br/indicador/leite.aspx'
})

csv_file = doc.entity('data:milk_price_beetwen_producers_brasil_csv', {
    'prov:label': 'CSV Data of milk price beetwen producets in Brasil for each region',
})

download_activity = doc.activity('project:download_and_save', other_attributes={
    'prov:label': 'Download and Save as CSV'
})

main_process_activity = doc.activity('project:script_de_processamento_leite.ipynb', other_attributes= {
    'prov:label': 'Data Manipulation Main Script'
})

dataframe_in_memory = doc.entity('data:dataframe_price_production', {
    'prov:label': 'DataFrame of Milk price to production',
    'prov:description': 'Dataframe generated by pandas storage in memory'
})


doc.used(main_process_activity, csv_file)

doc.used(download_activity, website_data)
doc.wasGeneratedBy(dataframe_in_memory, main_process_activity)
doc.wasGeneratedBy(csv_file, download_activity)

print(doc.get_provn())

In [ ]:
from prov.dot import prov_to_dot
dot = prov_to_dot(doc)
dot.write_png('milk_price_beetwen_producers.png')

## Dataset ipca Variation

In [ ]:
from prov.model import ProvDocument

doc = ProvDocument()
doc.add_namespace('data', 'http://example.org/mydata#')
doc.add_namespace('project', 'http://example.org/myproject#')
doc.add_namespace('inmet', 'https://bdmep.inmet.gov.br/#')

csv_file = doc.entity('data:ipca_variation_brasil_csv', {
    'prov:label': 'CSV Data of ipca variation in porcentage',
})
website_data = doc.entity('data:website_ipca_variation_brasil', {
    'prov:label': 'ipca variation in brasi;',
    'prov:location': 'https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=series-historicas'
})

dataframe_in_memory = doc.entity('data:dataframe_ipca_variation', {
    'prov:label': 'DataFrame of ipca variation',
    'prov:description': 'Dataframe generated by pandas in memory'
})

download_activity = doc.activity('project:download_and_save', other_attributes={
    'prov:label': 'Download and Save as CSV'
})

main_process_activity = doc.activity('project:script_de_processamento_leite.ipynb', other_attributes= {
    'prov:label': 'Data Manipulation Main Script'
})

doc.used(download_activity, website_data)
doc.wasGeneratedBy(csv_file, download_activity)

doc.used(main_process_activity, csv_file)
doc.wasGeneratedBy(dataframe_in_memory, main_process_activity)

print(doc.get_provn())

In [ ]:
from prov.dot import prov_to_dot
dot = prov_to_dot(doc)
dot.write_png('prov_ipca_variation_data.png')

# Sedes da embrapa pelo Brasil

In [ ]:
doc = ProvDocument()
doc.add_namespace('data', 'http://example.org/mydata#')
doc.add_namespace('project', 'http://example.org/myproject#')
doc.add_namespace('embrapa', 'https://www.embrapa.br/embrapa-no-brasil')

website_data = doc.entity('data:units_embrapa_brasil', {
    'prov:label': 'unitys of embrapa in Brasil',

})

download_activity = doc.activity('project:manually_get_data', other_attributes={
    'prov:label': 'Download and Save city and regions locations of embrapa unitys',
    'prov:description': 'manually scrapy headquarters location data from website',
    "prov:location": "https://www.embrapa.br/embrapa-no-brasil"
})

main_process_activity = doc.activity('project:leite_visualization_script.ipynb', other_attributes= {
    'prov:label': 'Data Manipulation Main Script'
})

map_data = doc.entity('project:visualization_data_map', other_attributes={
    'prov:label': 'data visualization',
    'prov:description': 'map for visualization data in geographic way'
})

doc.wasGeneratedBy(website_data, download_activity )

doc.used(main_process_activity, website_data)
doc.wasGeneratedBy(map_data, main_process_activity)

print(doc.get_provn())

dot = prov_to_dot(doc)
dot.write_png('map_embrapa_visualization.png')

# Sedes da Rede Brasileira de Qualidade de Leite

In [ ]:
doc = ProvDocument()
doc.add_namespace('data', 'http://example.org/mydata#')
doc.add_namespace('project', 'http://example.org/myproject#')
doc.add_namespace('RBQL', 'https://www.gov.br/agricultura/pt-br/assuntos/defesa-agropecuaria/laboratorios-credenciados/laboratorios-credenciados/produtos-de-origem-animal/rede-brasileira-de-qualidade-do-leite-rbql')

website_data = doc.entity('data:units_RBQL_brasil', {
    'prov:label': 'unitys of Rede Brasileira de Qualidade do Leite in Brasil',

})

download_activity = doc.activity('project:manually_get_data', other_attributes={
    'prov:label': 'Download and Save city and regions locations of RQBL unitys',
    'prov:description': 'manually scrapy headquarters location data from website',
    "prov:location": "https://www.gov.br/agricultura/pt-br/assuntos/defesa-agropecuaria/laboratorios-credenciados/laboratorios-credenciados/produtos-de-origem-animal/rede-brasileira-de-qualidade-do-leite-rbql"
})

main_process_activity = doc.activity('project:leite_visualization_script.ipynb', other_attributes= {
    'prov:label': 'Data Manipulation Main Script'
})

map_data = doc.entity('data:visualization_data_map', other_attributes={
    'prov:label': 'data visualization',
    'prov:description': 'map for visualization data in geographic way'
})

doc.wasGeneratedBy(website_data, download_activity )

doc.used(main_process_activity, website_data)
doc.wasGeneratedBy(map_data, main_process_activity)

print(doc.get_provn())

dot = prov_to_dot(doc)
dot.write_png('map_RBQL_visualization.png')